# BERT

At todays meeting, we're going to focus on BERT -- a model, which transforms our tokens into contextual word embeddings, that are of great quality!

We will use `transformers` library for that purpose, so let's install it first:

In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


#BERT

## Task 1: Tokenization using WordPiece

Below you can find a simple fragment of code, which downloads a `bert-base` model (an `uncased` version -- this model introduced a preprocessing step on the data that transformed each text into lowercased text) and instantiates appropriate tokenizer for that model. You can learn about this particular model here: https://huggingface.co/bert-base-uncased (I encourage you to read the description! Many models hosted on the huggingface website have great documentations and it's always worth checking them).

Then, in line 4, we define a text to be tokenized, run the tokenizer in line 5, and use the tokenized input as the input to BERT model, which is called in line 6. 

Please, run the code below. We generated BERT embeddings using 6 lines of code! ;)

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Runnning tokenizer by simply calling the tokenizer object (`tokenizer(text, return_tensors='pt')`) returns a dictionary containing 3 keys: `input_ids`, `token_type_ids`, and `attention_mask`. As for now, let's focus on `input_ids` only. 

You may visit the website: https://huggingface.co/docs/transformers/glossary to learn more about the role of `token_type_ids` and `attention_mask`.


The `input_ids` is a list of lists (represented as a tensor). The outer list collects documents, while the inner lists collect tokens in that document. Here, we processed only one document (sentence) so that there is only one "outer" list. 

Each of the inner lists contains a seqeunce of identifiers. These are the positions of tokens in the Vocabulary. They can be used to generate one-hot encoding representations for our words, since if we know the length of a vocabulary and know the position of a given token in the vocabulary, we can generate a vector of the vocabulary length that is filled with zeros, then we set the value assigned to a token position to 1 to generate one-hot encoding.

To produce the ids, we have to tokenize our text first. 

Moreover, those identifiers require less memory that storing tokens as strings!

Run the code below, to see the tokenizer's output. Please note that the number of token identifiers generated by the tokenizer is not equal to the number of tokens in the sequence. We'll see why is that in a minute.


In [4]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
print(len(text.split()), len(encoded_input['input_ids'][0]))

7 12


The ids generated by the tokenizer are not human readable. However, the tokenizer contains a mapping which relates known tokens to their positions, so that we can use it to map ids to tokens back!

The first line of code obtains the input ids defined for the first sequence provided. 
Then, we call `convert_ids_to_tokens` to transform those ids to tokens. Please run the code and analyze the output.

In [8]:
first_sentence_ids = encoded_input['input_ids'][0]
tokenizer.convert_ids_to_tokens(first_sentence_ids)

['[CLS]',
 'replace',
 'me',
 'by',
 'any',
 'text',
 'you',
 "'",
 'd',
 'like',
 '.',
 '[SEP]']

Whoa, we see that the tokenizer not only tokenizes our text (splits into tokens), but it also generated special tokens required by our model! Nice. Please note that after tokenization we don't have capital letters in our tokens. This is caused by the use of the `bert-base-uncased` model. Since the model was trained on lowercased data, the tokenizer also ensures that the tokens are lowercased.


## Subword units

However, the length of the generated `input_ids` may be even bigger as related to the length of our text. Sometimes, the vocabulary doesn't contain a given word as a whole. Since BERT used WordPiece tokenization, to handle such cases, the tokenizer tries to split those words into smaller fragments, that are stored in our vocabulary. Let's tokenize some document containing rare words and see the result.

In [9]:
tokenizer_output = tokenizer(['NVIDIA DGX A100'])

input_ids = tokenizer_output['input_ids'][0]
input_ids

[101, 1050, 17258, 2401, 1040, 2290, 2595, 17350, 8889, 102]

Whoa, we see that as a result of tokenization, we obtained many tokens. Let's see what they represent: 

In [10]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', 'n', '##vid', '##ia', 'd', '##g', '##x', 'a1', '##00', '[SEP]']

Ah, as we can see, the vocabulary assigned to `bert-base` doesn't contain tokens such as "nvidia", "dgx" and "a100", that's why they are split into subwrod units. 

Each time a given subword unit starts with a double hash (##), we know that this subword unit is a continuation of the previous token. 

We can use this information to reconstruct the original text, by joining those subword units (sometimes called subtokens) into full tokens. We can achieve this goal using the following line of code:

In [11]:
tokenizer.decode(input_ids)

2023-11-02 10:39:41.951895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 10:39:42.237543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 10:39:43.357245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'[CLS] nvidia dgx a100 [SEP]'

If we know how to map tokens to their positions in the vocabulary, the only missing part is to determine how long our one-hot-encoding vector should be (or how big is our vocabulary). 

This transformation of ids into one-hot-encodings is done automatically by the `transformer` library. However, you can check the size of the vocabulary easily using the following line of code:

In [12]:
tokenizer.vocab_size

30522

## Task 2: Using a pre-trained BERT to generate features that may be used to solve a classification task.

As we discussed during the lecture, we can generate a fixed-length vector represening any input by taking the embedding produced for the `[CLS]` token (which is a representation of the whole seqeunce).

In this task, your goal is to use a pre-trained BERT model to obtain representations for a given dataset. Then, these representations will be used to train a logistic regression model. 

We will try to genearate a solution that can detect whether a given review is positive or not!

For that purpose we're going to use a BERT variant called `distilBERT`. We didn't cover it during the lecture, because it is related to the concept of distillation that will be very important to us and I'll make a separate lecture on it. 

For now, we can treat `distilBERT` the same way as BERT. In fact, this is a compressed BERT model. It behaves the same way but it is distilled so that we aimed to achieve similar quality using less parameters.

Please follow the tutorial you can find here: https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb (there is even a blogpost related to this tutorial: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)


Just copy and paste fragments of code here and observe the results:

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [15]:
batch_1 = df[:2000] # cut dataset for performance reasons
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [17]:
# load pretrained distilbert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
model_class, tokenizer_class

(transformers.models.distilbert.modeling_distilbert.DistilBertModel,
 transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer)

In [19]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# pad to the same size so we can use them as batch
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

(2000, 59)

In [21]:
# say bert that we have padded tokens
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [22]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [23]:
# get embedding for [CLS] token, representing the whole sentence
features = last_hidden_states[0][:,0,:].numpy()
labels = batch_1[1] # positive or negative sentiment

In [24]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [28]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.8033333333333333


In [29]:
# train logistic regression on top of bert embeddings

lr_clf = LogisticRegression(C=5.263157894736842)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.263157894736842)

In [30]:
lr_clf.score(test_features, test_labels)

0.836

In [31]:
# get dummy classifier score as a baseline

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.521 (+/- 0.00)


## Task 3(Optional - not required): fine-tuning BERT

In the example above, a BERT model (or more precisely it's smaller family member: distilBERT) was used only to deliver vectors representing whole documents. 

Now we would like to fine-tune an existing model. As we discussed during the lecture, we can achieve it by simply switching the top-layer of the network. Instead of solving the Masked Language Model and Next Sentence Prediction tasks, we may add our own classification layer (also referred to as a classification head) and train our whole network to solve a given task.

There is a great and easy to follow tutorial on fine-tuning, available here: https://github.com/huggingface/notebooks/blob/main/transformers_doc/training.ipynb 

If you want, please follow that tutorial (you can copy-and-paste the code snippets from the notebook here).

In [17]:
### -------------------------------------------------------------
### a placeholder for code copied-and-pasted from the tutorial ;)
### -------------------------------------------------------------